In [1]:
import scipy.io as spio
import numpy as np
from utilities import *
from sklearn.model_selection import train_test_split

In [10]:
# Input data
x_mat = spio.loadmat(file_name='../data/bci-sample-data/x.mat')
print(x_mat.keys())
x_data = x_mat['Intensification_Data']
print(x_data.shape, x_data.dtype)

dict_keys(['__header__', '__version__', '__globals__', 'Intensification_Data'])
(6480, 192, 64) float64


In [45]:
# Output data: class labels
y_mat = spio.loadmat(file_name='../data/bci-sample-data/y.mat')
print(y_mat.keys(), y_mat['Intensification_SType'].shape)
y_labels = np.mean(y_mat['Intensification_SType'], axis=1, dtype=int)
print(y_labels.shape, y_labels.dtype)
print(np.mean(y_labels==0), np.mean(y_labels==1))

dict_keys(['__header__', '__version__', '__globals__', 'Intensification_SType']) (6480, 192)
(6480,) int64
0.833333333333 0.166666666667


In [46]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_data,
                                                      y_labels,
                                                      stratify = y_labels,
                                                      random_state = 123,
                                                      test_size=0.30)
print(X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape)
print(X_train.dtype, X_valid.dtype, Y_train.dtype, Y_valid.dtype)

(4536, 192, 64) (1944, 192, 64) (4536,) (1944,)
float64 float64 int64 int64


In [47]:
# Standardize/normalize train and test
X_train_norm, X_valid_norm = standardize(train=X_train, test=X_valid)

print(X_train_norm.shape, X_train_norm.dtype, 
X_valid_norm.shape, X_valid_norm.dtype)

(4536, 192, 64) float64 (1944, 192, 64) float64


In [48]:
### Hyperparameters

# Input data
batch_size = X_train_norm.shape[0]// 100 # minibatch size & number of minibatches
seq_len = X_train_norm.shape[1] # Number of steps: each trial length
n_channels = X_train_norm.shape[2] # number of channels in each trial
print('batch_size, seq_len, n_channels', batch_size, seq_len, n_channels)

# Output labels
n_classes = int(y_labels[0].max(axis=0)+1)
print('n_classes', n_classes)

# learning parameters
learning_rate = 0.0001 #1e-4
epochs = 100 # num iterations for updating model
keep_prob = 0.50 # 90% neurons are kept and 10% are dropped out

batch_size, seq_len, n_channels 45 192 64
n_classes 2


In [49]:
Y_train_onehot = one_hot(labels=Y_train, n_class=n_classes)
Y_valid_onehot = one_hot(labels=Y_valid, n_class=n_classes)

print(Y_train_onehot.shape, Y_valid_onehot.shape, 
 X_train_norm.shape, X_valid_norm.shape)

(4536, 2) (1944, 2) (4536, 192, 64) (1944, 192, 64)


In [50]:
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [51]:
#  Buffering/ placeholders to transfer the data from py to tf
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs_')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels_')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep_prob_')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate_')# Construct the LSTM inputs and LSTM cells

In [53]:
# with graph.as_default():
# (451, 3072, 13) float64 (193, 3072, 13) float64 (276, 3072, 13) float64
# (4536, 192, 64) (1944, 192, 64) (4536,) (1944,)
# (batch, 192, 64) --> (batch, 96, 128)
conv1 = tf.layers.conv1d(inputs=inputs_, filters=128, kernel_size=2, strides=1, padding='same', 
                         activation = tf.nn.relu)
max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
# max_pool_1 = tf.nn.dropout(max_pool_1, keep_prob=keep_prob_)
print(conv1.shape, max_pool_1.shape)

# (batch, 96, 128) --> (batch, 48, 256)
conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=256, kernel_size=2, strides=1, padding='same', 
                         activation = tf.nn.relu)
max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
# max_pool_2 = tf.nn.dropout(max_pool_2, keep_prob=keep_prob_)
print(conv2.shape, max_pool_2.shape)

# (batch, 48, 256) --> (batch, 24, 512)
conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=512, kernel_size=2, strides=1, padding='same', 
                         activation = tf.nn.relu)
max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
# max_pool_3 = tf.nn.dropout(max_pool_3, keep_prob=keep_prob_)
print(conv3.shape, max_pool_3.shape)

# (batch, 24, 512) --> (batch, 12, 1024)
conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=1024, kernel_size=2, strides=1, padding='same', 
                         activation = tf.nn.relu)
max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')
# max_pool_4 = tf.nn.dropout(max_pool_4, keep_prob=keep_prob_)
print(conv4.shape, max_pool_4.shape)

(?, 192, 128) (?, 96, 128)
(?, 96, 256) (?, 48, 256)
(?, 48, 512) (?, 24, 512)
(?, 24, 1024) (?, 12, 1024)


In [56]:
# Flatten and add dropout + predicted output
flat = tf.reshape(max_pool_4, (-1, 12*1024))
flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
logits = tf.layers.dense(flat, n_classes)
print(max_pool_4.shape, flat.shape, logits.shape)

(?, 12, 1024) (?, 12288) (?, 2)


In [57]:
# Backward pass: error backpropagation
# Cost function
cost_tensor = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_)
cost = tf.reduce_mean(input_tensor=cost_tensor)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
print(correct_pred, accuracy)

Tensor("Equal:0", shape=(?,), dtype=bool) Tensor("accuracy:0", shape=(), dtype=float32)


In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

# Save the training result or trained and validated model params
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x, y in get_batches(X_train_norm, Y_train_onehot, batch_size):
            
            ######################## Training
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : keep_prob, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            ################## Validation
            val_acc_ = []
            val_loss_ = []    
            for x_v, y_v in get_batches(X_valid_norm, Y_valid_onehot, batch_size):
                
                # Feed
                feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  

                # Loss
                loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)

            # Store
            validation_acc.append(np.mean(val_acc_))
            validation_loss.append(np.mean(val_loss_))
            
        # Print info for every iter/epoch
        print("Epoch: {}/{}".format(e+1, epochs),
              "Train loss: {:6f}".format(np.mean(train_loss)),
              "Valid loss: {:.6f}".format(np.mean(validation_loss)),
              "Train acc: {:6f}".format(np.mean(train_acc)),
              "Valid acc: {:.6f}".format(np.mean(validation_acc)))
                
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 1/100 Train loss: 0.471488 Valid loss: 0.473677 Train acc: 0.829778 Valid acc: 0.833075
Epoch: 2/100 Train loss: 0.456434 Valid loss: 0.466009 Train acc: 0.831333 Valid acc: 0.833070
Epoch: 3/100 Train loss: 0.441879 Valid loss: 0.456761 Train acc: 0.832074 Valid acc: 0.833116
Epoch: 4/100 Train loss: 0.424683 Valid loss: 0.444778 Train acc: 0.836556 Valid acc: 0.833836
Epoch: 5/100 Train loss: 0.408135 Valid loss: 0.433275 Train acc: 0.841467 Valid acc: 0.835527
Epoch: 6/100 Train loss: 0.391972 Valid loss: 0.423798 Train acc: 0.847407 Valid acc: 0.837553
Epoch: 7/100 Train loss: 0.376409 Valid loss: 0.415873 Train acc: 0.853270 Valid acc: 0.839561
Epoch: 8/100 Train loss: 0.361936 Valid loss: 0.409821 Train acc: 0.858528 Valid acc: 0.841142
Epoch: 9/100 Train loss: 0.347894 Valid loss: 0.405094 Train acc: 0.864025 Valid acc: 0.842585
Epoch: 10/100 Train loss: 0.334930 Valid loss: 0.401202 Train acc: 0.869156 Valid acc: 0.843800
Epoch: 11/100 Train loss: 0.322238 Valid loss: 0.

In [ ]:
import matplotlib.pyplot as mplot

mplot.plot(train_loss, label='train_loss')
mplot.plot(validation_loss, label='valid_loss')
mplot.legend()
mplot.show()

In [ ]:
# import matplotlib.pyplot as mplot

mplot.plot(train_acc, label='train_acc')
mplot.plot(validation_acc, label='valid_acc')
mplot.legend()
mplot.show()

In [19]:
# test_acc = []

# with tf.Session() as sess:
#     # Restore
#     saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
#     for x_t, y_t in get_batches(X_test_norm, Y_test_onehot, batch_size):
#         feed = {inputs_: x_t,
#                 labels_: y_t,
#                 keep_prob_: 1}
        
#         batch_acc = sess.run(accuracy, feed_dict=feed)
#         test_acc.append(batch_acc)
#     print("Test accuracy: {:.6f}".format(np.mean(test_acc)))